# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

In [ ]:
# import libraries
import pyspark
from pyspark.sql import SparkSession, Row
import pyspark.sql.functions as F
import pyspark.sql.types as T

from pyspark.ml import Pipeline, PipelineModel
import pyspark.ml.classification as cl
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
import pyspark.ml.feature as ft
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

import re
import time

In [ ]:
pyspark.__version__

'3.4.1'

In [ ]:
# create a Spark session
spark = SparkSession \
    .builder \
    .appName("Sparkify") \
    .getOrCreate()
spark.sparkContext.getConf().getAll()

[('spark.app.id', 'local-1687856550614'),
 ('spark.app.startTime', '1687856546239'),
 ('spark.driver.host', '7457b600f8b8'),
 ('spark.executor.id', 'driver'),
 ('spark.app.submitTime', '1687856545770'),
 ('spark.driver.extraJavaOptions',
  '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNA

In [ ]:
spark

# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids.

In [ ]:
path = "mini_sparkify_event_data.json"
dataset = spark.read.json(path)

In [ ]:
dataset.printSchema()

root
 |-- _corrupt_record: string (nullable = true)
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [ ]:
dataset.take(3)

[Row(_corrupt_record=None, artist='Martha Tilston', auth='Logged In', firstName='Colin', gender='M', itemInSession=50, lastName='Freeman', length=277.89016, level='paid', location='Bakersfield, CA', method='PUT', page='NextSong', registration=1538173362000, sessionId=29, song='Rockpools', status=200, ts=1538352117000, userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0', userId='30'),
 Row(_corrupt_record=None, artist='Five Iron Frenzy', auth='Logged In', firstName='Micah', gender='M', itemInSession=79, lastName='Long', length=236.09424, level='free', location='Boston-Cambridge-Newton, MA-NH', method='PUT', page='NextSong', registration=1538331630000, sessionId=8, song='Canada', status=200, ts=1538352180000, userAgent='"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.103 Safari/537.36"', userId='9'),
 Row(_corrupt_record=None, artist='Adam Lambert', auth='Logged In', firstName='Colin', gender='M', itemInSession

In [ ]:
cols=['userId', 'page','location','userAgent','gender','artist','song']

In [ ]:
dataset=dataset.select(cols)

In [ ]:
dataset=dataset.dropna(how = "any", subset = ["userId",])

In [ ]:
dataset=dataset.fillna("None")

In [ ]:
isdigit=F.udf(lambda str:str.isdigit(),T.BooleanType())

In [ ]:
dataset=dataset.filter(isdigit(dataset.userId))

In [ ]:
dataset.count()

59568

Set the type of `usedId` to integer.

In [ ]:
dataset=dataset.withColumn("userId",dataset.userId.cast('long'))

In [ ]:
dataset.persist()

DataFrame[userId: bigint, page: string, location: string, userAgent: string, gender: string, artist: string, song: string]

# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

In [ ]:
dataset.select('page').distinct().show()

+--------------------+
|                page|
+--------------------+
|              Cancel|
|    Submit Downgrade|
|         Thumbs Down|
|                Home|
|           Downgrade|
|         Roll Advert|
|              Logout|
|       Save Settings|
|Cancellation Conf...|
|               About|
|            Settings|
|     Add to Playlist|
|          Add Friend|
|            NextSong|
|           Thumbs Up|
|                Help|
|             Upgrade|
|               Error|
|      Submit Upgrade|
+--------------------+



In order to look for the `churn`s, we will group the data by `userId` and collect all the `page`s as `history`.

In [ ]:
history=dataset.select(['userId','page']).groupby('userId').agg(F.collect_list('page').alias('history'))

In [ ]:
history.show()

+------+--------------------+
|userId|             history|
+------+--------------------+
|     2|[NextSong, NextSo...|
|     3|[NextSong, NextSo...|
|     4|[NextSong, NextSo...|
|     5|[NextSong, NextSo...|
|     6|[NextSong, NextSo...|
|     7|[NextSong, NextSo...|
|     8|[Home, Add Friend...|
|     9|[Home, NextSong, ...|
|    10|[NextSong, NextSo...|
|    11|[NextSong, Roll A...|
|    12|[Home, NextSong, ...|
|    13|[NextSong, Roll A...|
|    14|[NextSong, NextSo...|
|    15|[Home, NextSong, ...|
|    16|[NextSong, Add to...|
|    17|[NextSong, NextSo...|
|    18|[Home, NextSong, ...|
|    19|[NextSong, NextSo...|
|    20|[Home, NextSong, ...|
|    21|[Home, NextSong, ...|
+------+--------------------+
only showing top 20 rows



Then, we define `churn` to be `1` if `{'Cancellation Confirmation','Submit Downgrade'}` is subset of the `history` and `0` otherwise.

In [ ]:
issubset = F.udf(lambda l: int('Cancellation Confirmation' in l), T.IntegerType())

In [ ]:
history=history.withColumn("churn",issubset(history.history))

In [ ]:
history.show()

+------+--------------------+-----+
|userId|             history|churn|
+------+--------------------+-----+
|     2|[NextSong, NextSo...|    0|
|     3|[NextSong, NextSo...|    0|
|     4|[NextSong, NextSo...|    0|
|     5|[NextSong, NextSo...|    0|
|     6|[NextSong, NextSo...|    0|
|     7|[NextSong, NextSo...|    0|
|     8|[Home, Add Friend...|    0|
|     9|[Home, NextSong, ...|    0|
|    10|[NextSong, NextSo...|    0|
|    11|[NextSong, Roll A...|    0|
|    12|[Home, NextSong, ...|    0|
|    13|[NextSong, Roll A...|    0|
|    14|[NextSong, NextSo...|    0|
|    15|[Home, NextSong, ...|    0|
|    16|[NextSong, Add to...|    0|
|    17|[NextSong, NextSo...|    1|
|    18|[Home, NextSong, ...|    1|
|    19|[NextSong, NextSo...|    0|
|    20|[Home, NextSong, ...|    0|
|    21|[Home, NextSong, ...|    0|
+------+--------------------+-----+
only showing top 20 rows



We compute now the length of the history as an indicator of user activity.

In [ ]:
Len=F.udf(len,T.IntegerType())

In [ ]:
history=history.withColumn("nHist",Len("history"))

In [ ]:
history.show()

+------+--------------------+-----+-----+
|userId|             history|churn|nHist|
+------+--------------------+-----+-----+
|     2|[NextSong, NextSo...|    0|  428|
|     3|[NextSong, NextSo...|    0|   24|
|     4|[NextSong, NextSo...|    0|  110|
|     5|[NextSong, NextSo...|    0|   41|
|     6|[NextSong, NextSo...|    0|  586|
|     7|[NextSong, NextSo...|    0|   51|
|     8|[Home, Add Friend...|    0|  155|
|     9|[Home, NextSong, ...|    0|  946|
|    10|[NextSong, NextSo...|    0|  421|
|    11|[NextSong, Roll A...|    0|  140|
|    12|[Home, NextSong, ...|    0|  971|
|    13|[NextSong, Roll A...|    0|  422|
|    14|[NextSong, NextSo...|    0|   48|
|    15|[Home, NextSong, ...|    0| 1189|
|    16|[NextSong, Add to...|    0|  435|
|    17|[NextSong, NextSo...|    1| 1102|
|    18|[Home, NextSong, ...|    1|  513|
|    19|[NextSong, NextSo...|    0|  259|
|    20|[Home, NextSong, ...|    0|  298|
|    21|[Home, NextSong, ...|    0|   56|
+------+--------------------+-----

In [ ]:
history.agg(F.min("nHist"),F.max("nHist")).show()

+----------+----------+
|min(nHist)|max(nHist)|
+----------+----------+
|         1|      2850|
+----------+----------+



In [ ]:
history=history.drop("nHist")

We look for the person `trajectory`, we will group the data by `userId` and collect all the `location`s.

In [ ]:
trajectory=dataset.select(['userId','location']).groupby('userId').agg(F.collect_list('location').alias('trajectory'))

In [ ]:
lenSet=F.udf(lambda x:len(set(x)),T.IntegerType())

In [ ]:
trajectory=trajectory.withColumn("nLoc",lenSet("trajectory"))

In [ ]:
trajectory.agg(F.max("nLoc")).show()

+---------+
|max(nLoc)|
+---------+
|        1|
+---------+



We have only one location person. So we reduce all the list for the first element.

In [ ]:
trajectory=trajectory.drop("nLoc")

In [ ]:
reduction=F.udf(lambda x:x[0])

In [ ]:
trajectory=trajectory.withColumn("location",reduction("trajectory")).drop("trajectory")

In [ ]:
trajectory.show()

+------+--------------------+
|userId|            location|
+------+--------------------+
|     2|         Raleigh, NC|
|     3|         Bozeman, MT|
|     4|Baltimore-Columbi...|
|     5|Phoenix-Mesa-Scot...|
|     6|Houston-The Woodl...|
|     7|Seattle-Tacoma-Be...|
|     8|    St. Louis, MO-IL|
|     9|Boston-Cambridge-...|
|    10|          Laurel, MS|
|    11|Providence-Warwic...|
|    12|Los Angeles-Long ...|
|    13|Philadelphia-Camd...|
|    14|        Syracuse, NY|
|    15|      Morgantown, WV|
|    16|   Winston-Salem, NC|
|    17|Philadelphia-Camd...|
|    18|  Kansas City, MO-KS|
|    19|          Dayton, OH|
|    20|Miami-Fort Lauder...|
|    21|        Kankakee, IL|
+------+--------------------+
only showing top 20 rows



In [ ]:
trajectory.select('location').distinct().count()

88

We fo the same for `userAgent`.

In [ ]:
userAgents=dataset.select(['userId','userAgent']).groupby('userId').agg(F.collect_list('userAgent').alias('userAgents'))

In [ ]:
userAgents=userAgents.withColumn("nAgent",lenSet("userAgents"))

In [ ]:
userAgents.agg(F.max("nAgent")).show()

+-----------+
|max(nAgent)|
+-----------+
|          1|
+-----------+



We have only one browser person. So we reduce all the list for the first element.

In [ ]:
userAgents=userAgents.drop("nAgent")

In [ ]:
userAgents=userAgents.withColumn("userAgent",reduction("userAgents")).drop("userAgents")

In [ ]:
userAgents.show()

+------+--------------------+
|userId|           userAgent|
+------+--------------------+
|     2|"Mozilla/5.0 (Win...|
|     3|"Mozilla/5.0 (Win...|
|     4|Mozilla/5.0 (Wind...|
|     5|"Mozilla/5.0 (Win...|
|     6|"Mozilla/5.0 (Mac...|
|     7|Mozilla/5.0 (Wind...|
|     8|"Mozilla/5.0 (Win...|
|     9|"Mozilla/5.0 (Win...|
|    10|"Mozilla/5.0 (Mac...|
|    11|Mozilla/5.0 (Wind...|
|    12|Mozilla/5.0 (Wind...|
|    13|"Mozilla/5.0 (iPh...|
|    14|Mozilla/5.0 (Maci...|
|    15|"Mozilla/5.0 (Win...|
|    16|"Mozilla/5.0 (Win...|
|    17|Mozilla/5.0 (Wind...|
|    18|"Mozilla/5.0 (Mac...|
|    19|Mozilla/5.0 (Wind...|
|    20|"Mozilla/5.0 (Win...|
|    21|Mozilla/5.0 (Wind...|
+------+--------------------+
only showing top 20 rows



In [ ]:
sub=F.udf(lambda x:re.sub('"',"",x),T.StringType())

In [ ]:
userAgents=userAgents.withColumn("userAgent",sub("userAgent"))

In [ ]:
userAgents.select('userAgent').distinct().count()

46

We continue with same logic for the `gender`.

In [ ]:
dataset.select('gender').distinct().count()

2

In [ ]:
genders=dataset.select(['userId','gender']).groupby('userId').agg(F.collect_list('gender').alias('genders'))

In [ ]:
genders=genders.withColumn("nGender",lenSet("genders"))

In [ ]:
genders.agg(F.max("nGender")).show()

+------------+
|max(nGender)|
+------------+
|           1|
+------------+



In [ ]:
genders=genders.drop("nGender")

In [ ]:
genders=genders.withColumn("gender",reduction("genders")).drop("genders")

In [ ]:
genders.show()

+------+------+
|userId|gender|
+------+------+
|     2|     F|
|     3|     M|
|     4|     M|
|     5|     M|
|     6|     M|
|     7|     M|
|     8|     F|
|     9|     M|
|    10|     M|
|    11|     F|
|    12|     F|
|    13|     F|
|    14|     M|
|    15|     M|
|    16|     F|
|    17|     F|
|    18|     M|
|    19|     F|
|    20|     F|
|    21|     M|
+------+------+
only showing top 20 rows



For the artist and song, we will keep the wole list.

In [ ]:
dataset.select('artist').distinct().count()

9623

In [ ]:
artists=dataset.select(['userId','artist']).groupby('userId').agg(F.collect_list('artist').alias('artists'))

In [ ]:
artists=artists.withColumn("nArtist",lenSet("artists"))

In [ ]:
artists.agg(F.max("nArtist")).show()

+------------+
|max(nArtist)|
+------------+
|        1524|
+------------+



In [ ]:
artists=artists.drop("nArtist")

In [ ]:
artists.show()

+------+--------------------+
|userId|             artists|
+------+--------------------+
|     2|[Al Di Meola, Kar...|
|     3|[Orenda Fink, Bar...|
|     4|[Third Day, Littl...|
|     5|[Joan Baez, Metri...|
|     6|[Cobra Starship F...|
|     7|[Crystal Castles,...|
|     8|[None, None, None...|
|     9|[None, Eminem, Ju...|
|    10|[Alexis & Fido, C...|
|    11|[Carly Simon, Non...|
|    12|[None, Audioslave...|
|    13|[The Kills, None,...|
|    14|[Muchachito Bombo...|
|    15|[None, Cradle Of ...|
|    16|[Freddie King, No...|
|    17|[Grant Lee Buffal...|
|    18|[None, Mike And T...|
|    19|[Red, Verona, Sea...|
|    20|[None, Eric Clapt...|
|    21|[None, Lonnie Gor...|
+------+--------------------+
only showing top 20 rows



In [ ]:
dataset.select('song').distinct().count()

23371

In [ ]:
songs=dataset.select(['userId','song']).groupby('userId').agg(F.collect_list('song').alias('songs'))

In [ ]:
songs=songs.withColumn("nSong",lenSet("songs"))

In [ ]:
songs.agg(F.max("nSong")).show()

+----------+
|max(nSong)|
+----------+
|      2057|
+----------+



In [ ]:
songs=songs.drop("nSong")

In [ ]:
songs.show()

+------+--------------------+
|userId|               songs|
+------+--------------------+
|     2|[Roller Jubilee, ...|
|     3|[Miracle Worker, ...|
|     4|[Mama, Earthquake...|
|     5|[Wildwood Flower,...|
|     6|[Good Girls Go Ba...|
|     7|[Not In Love, Yea...|
|     8|[None, None, None...|
|     9|[None, Without Me...|
|    10|[Bartender, Suite...|
|    11|[He Likes To Roll...|
|    12|[None, Drown Me S...|
|    13|[Getting Down, No...|
|    14|[Luna, Exterminat...|
|    15|[None, Better To ...|
|    16|[Just A Little Bi...|
|    17|[The Hook, Pens A...|
|    18|[None, A Beggar O...|
|    19|[Ordinary World, ...|
|    20|[None, Kind Heart...|
|    21|[None, Catch You ...|
+------+--------------------+
only showing top 20 rows



Now, we combine all together.

In [ ]:
data=history.orderBy('userId')
for temp in [trajectory,userAgents,genders,artists,songs]:
    data=data.join(temp,'userId')

In [ ]:
data.show(vertical=True)

-RECORD 0-------------------------
 userId    | 2                    
 history   | [NextSong, NextSo... 
 churn     | 0                    
 location  | Raleigh, NC          
 userAgent | Mozilla/5.0 (Wind... 
 gender    | F                    
 artists   | [Al Di Meola, Kar... 
 songs     | [Roller Jubilee, ... 
-RECORD 1-------------------------
 userId    | 3                    
 history   | [NextSong, NextSo... 
 churn     | 0                    
 location  | Bozeman, MT          
 userAgent | Mozilla/5.0 (Wind... 
 gender    | M                    
 artists   | [Orenda Fink, Bar... 
 songs     | [Miracle Worker, ... 
-RECORD 2-------------------------
 userId    | 4                    
 history   | [NextSong, NextSo... 
 churn     | 0                    
 location  | Baltimore-Columbi... 
 userAgent | Mozilla/5.0 (Wind... 
 gender    | M                    
 artists   | [Third Day, Littl... 
 songs     | [Mama, Earthquake... 
-RECORD 3-------------------------
 userId    | 5      

In [ ]:
data.groupby("churn").count().show()

+-----+-----+
|churn|count|
+-----+-----+
|    1|   10|
|    0|  125|
+-----+-----+



In [ ]:
data.groupby("gender").count().show()

+------+-----+
|gender|count|
+------+-----+
|     F|   63|
|     M|   72|
+------+-----+



In [ ]:
data.groupby(["churn","gender"]).count().show()

+-----+------+-----+
|churn|gender|count|
+-----+------+-----+
|    1|     F|    2|
|    0|     M|   64|
|    1|     M|    8|
|    0|     F|   61|
+-----+------+-----+



In [ ]:
data.groupby("location").count().show()

+--------------------+-----+
|            location|count|
+--------------------+-----+
|     Gainesville, FL|    1|
|Atlantic City-Ham...|    1|
|Deltona-Daytona B...|    1|
|San Diego-Carlsba...|    1|
|Kingsport-Bristol...|    1|
|New Haven-Milford...|    2|
|  Corpus Christi, TX|    1|
|         Dubuque, IA|    1|
|Las Vegas-Henders...|    3|
|Indianapolis-Carm...|    2|
|Seattle-Tacoma-Be...|    2|
|   Winston-Salem, NC|    1|
|Los Angeles-Long ...|    9|
|     Bakersfield, CA|    1|
|Minneapolis-St. P...|    1|
|San Francisco-Oak...|    1|
|Phoenix-Mesa-Scot...|    5|
|Allentown-Bethleh...|    1|
|Miami-Fort Lauder...|    2|
|          Dayton, OH|    1|
+--------------------+-----+
only showing top 20 rows



In [ ]:
data.groupby(["churn","location"]).count().show()

+-----+--------------------+-----+
|churn|            location|count|
+-----+--------------------+-----+
|    0|Seattle-Tacoma-Be...|    2|
|    0|    St. Louis, MO-IL|    2|
|    0|      Morgantown, WV|    1|
|    1|           Flint, MI|    1|
|    0|San Diego-Carlsba...|    1|
|    0|       Fairbanks, AK|    1|
|    0|San Francisco-Oak...|    1|
|    0|Minneapolis-St. P...|    1|
|    0|         Bozeman, MT|    1|
|    0|Los Angeles-Long ...|    9|
|    0|        Syracuse, NY|    1|
|    0|Spokane-Spokane V...|    1|
|    0|New York-Newark-J...|    7|
|    0|Riverside-San Ber...|    1|
|    0|      Montgomery, AL|    1|
|    0|Kingsport-Bristol...|    1|
|    0|Indianapolis-Carm...|    1|
|    0|Las Vegas-Henders...|    3|
|    1|Philadelphia-Camd...|    1|
|    1|Oxnard-Thousand O...|    1|
+-----+--------------------+-----+
only showing top 20 rows



In [ ]:
data.groupby("userAgent").count().show()

+--------------------+-----+
|           userAgent|count|
+--------------------+-----+
|Mozilla/5.0 (Wind...|   16|
|Mozilla/5.0 (Wind...|    5|
|Mozilla/5.0 (Maci...|    1|
|Mozilla/5.0 (X11;...|    1|
|Mozilla/5.0 (X11;...|    1|
|Mozilla/5.0 (Wind...|    1|
|Mozilla/5.0 (Maci...|    1|
|Mozilla/5.0 (Maci...|   10|
|Mozilla/5.0 (Wind...|    1|
|Mozilla/5.0 (Wind...|    4|
|Mozilla/5.0 (comp...|    2|
|Mozilla/5.0 (Maci...|    3|
|Mozilla/5.0 (Wind...|    3|
|Mozilla/5.0 (Wind...|   15|
|Mozilla/5.0 (Maci...|    1|
|Mozilla/5.0 (X11;...|    1|
|Mozilla/5.0 (comp...|    1|
|Mozilla/5.0 (Maci...|    1|
|Mozilla/5.0 (Maci...|    1|
|Mozilla/5.0 (Wind...|    1|
+--------------------+-----+
only showing top 20 rows



In [ ]:
data.groupby(["churn","userAgent"]).count().show()

+-----+--------------------+-----+
|churn|           userAgent|count|
+-----+--------------------+-----+
|    0|Mozilla/5.0 (Wind...|    1|
|    0|Mozilla/5.0 (iPho...|    1|
|    0|Mozilla/5.0 (Maci...|    1|
|    0|Mozilla/5.0 (Wind...|    5|
|    0|Mozilla/5.0 (X11;...|    1|
|    0|Mozilla/5.0 (Wind...|    1|
|    0|Mozilla/5.0 (Maci...|    1|
|    0|Mozilla/5.0 (Maci...|    1|
|    0|Mozilla/5.0 (Wind...|    4|
|    0|Mozilla/5.0 (Maci...|    7|
|    0|Mozilla/5.0 (Wind...|    5|
|    0|Mozilla/5.0 (comp...|    2|
|    0|Mozilla/5.0 (Wind...|    1|
|    0|Mozilla/5.0 (Wind...|    2|
|    1|Mozilla/5.0 (Wind...|    1|
|    0|Mozilla/5.0 (Maci...|    9|
|    0|Mozilla/5.0 (Maci...|    1|
|    0|Mozilla/5.0 (Maci...|    1|
|    0|Mozilla/5.0 (iPho...|    1|
|    0|Mozilla/5.0 (comp...|    1|
+-----+--------------------+-----+
only showing top 20 rows



# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

In [ ]:
@F.udf(returnType=T.ArrayType(T.StringType()))
def remove(x):

    while True:
        try:
            x.remove('Cancellation Confirmation')
        except ValueError:
            break

    return x

In [ ]:
data=data.withColumn("history",remove("history"))

In [ ]:
data.show(vertical=True,n=3)

-RECORD 0-------------------------
 userId    | 2                    
 history   | [NextSong, NextSo... 
 churn     | 0                    
 location  | Raleigh, NC          
 userAgent | Mozilla/5.0 (Wind... 
 gender    | F                    
 artists   | [Al Di Meola, Kar... 
 songs     | [Roller Jubilee, ... 
-RECORD 1-------------------------
 userId    | 3                    
 history   | [NextSong, NextSo... 
 churn     | 0                    
 location  | Bozeman, MT          
 userAgent | Mozilla/5.0 (Wind... 
 gender    | M                    
 artists   | [Orenda Fink, Bar... 
 songs     | [Miracle Worker, ... 
-RECORD 2-------------------------
 userId    | 4                    
 history   | [NextSong, NextSo... 
 churn     | 0                    
 location  | Baltimore-Columbi... 
 userAgent | Mozilla/5.0 (Wind... 
 gender    | M                    
 artists   | [Third Day, Littl... 
 songs     | [Mama, Earthquake... 
only showing top 3 rows



In [ ]:
data.write.mode("overwrite").json("traited_data.json")

In [ ]:
!zip -u -r data.zip traited_data.json

	zip warning: data.zip not found or empty
  adding: traited_data.json/ (stored 0%)
  adding: traited_data.json/.part-00000-f7b9c656-81a3-403e-abb9-7167c9bf8594-c000.json.crc (stored 0%)
  adding: traited_data.json/part-00000-f7b9c656-81a3-403e-abb9-7167c9bf8594-c000.json (deflated 69%)
  adding: traited_data.json/_SUCCESS (stored 0%)
  adding: traited_data.json/._SUCCESS.crc (stored 0%)


# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

In [ ]:
!unzip -u data.zip

Archive:  data.zip


In [ ]:
data=spark.read.json("traited_data.json")

In [ ]:
cvHist = ft.CountVectorizer(inputCol="history", outputCol="cvHistory")
idfHist = ft.IDF(inputCol="cvHistory", outputCol="idfHistory")

In [ ]:
userAgentW = ft.Tokenizer(inputCol="userAgent", outputCol="userAgentWords")
cvUserAgent = ft.CountVectorizer(inputCol="userAgentWords",outputCol="userAgentCV")
idfUserAgent = ft.IDF(inputCol="userAgentCV",outputCol="userAgentIDF")

In [ ]:
cvArtist = ft.CountVectorizer(inputCol="artists", outputCol="artistCV")
idfArtist = ft.IDF(inputCol="artistCV", outputCol="artistIDF")

In [ ]:
cvSong = ft.CountVectorizer(inputCol="songs", outputCol="songCV")
idfSong = ft.IDF(inputCol="songCV", outputCol="songIDF")

In [ ]:
stGender = ft.StringIndexer(inputCol="gender", outputCol="indexGender")
HotGender = ft.OneHotEncoder(inputCol="indexGender", outputCol="hotGender")

In [ ]:
stLoc = ft.StringIndexer(inputCol="location", outputCol="indexLocation", handleInvalid="keep")
HotLoc = ft.OneHotEncoder(inputCol="indexLocation", outputCol="hotLocation")

In [ ]:
vectAssembler = ft.VectorAssembler(inputCols=["idfHistory","hotLocation", "userAgentIDF",
                                              "songIDF","songCV", "hotGender"],
                              outputCol="features")

In [ ]:
clf=cl.RandomForestClassifier(labelCol="churn")

In [ ]:
pipeline= Pipeline(stages=[cvHist, idfHist,
                           cvArtist, idfArtist,
                           cvSong, idfSong,
                           stGender, HotGender,
                           stLoc, HotLoc,
                           userAgentW, cvUserAgent, idfUserAgent,
                           vectAssembler,
                           clf
                         ]
                 )

In [ ]:
evaluator=MulticlassClassificationEvaluator(labelCol="churn",metricName="f1")

In [ ]:
train, test =data.randomSplit([0.8,0.2])

In [ ]:
paramGrid= ParamGridBuilder() \
    .baseOn({clf.numTrees:1000}) \
    .addGrid(clf.maxDepth,list(range(5,31,5))) \
    .build()

In [ ]:
crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                         )

In [ ]:
dt=time.time()

In [ ]:
cvModel=crossval.fit(train)

In [ ]:
(time.time()-dt)/3600

0.12036259664429559

In [ ]:
cvModel.bestModel.write().overwrite().save("bestModel")

In [ ]:
!zip -u -r bestModel.zip bestModel

	zip warning: bestModel.zip not found or empty
  adding: bestModel/ (stored 0%)
  adding: bestModel/metadata/ (stored 0%)
  adding: bestModel/metadata/part-00000 (deflated 39%)
  adding: bestModel/metadata/.part-00000.crc (stored 0%)
  adding: bestModel/metadata/_SUCCESS (stored 0%)
  adding: bestModel/metadata/._SUCCESS.crc (stored 0%)
  adding: bestModel/stages/ (stored 0%)
  adding: bestModel/stages/14_RandomForestClassifier_06e8be3d9d94/ (stored 0%)
  adding: bestModel/stages/14_RandomForestClassifier_06e8be3d9d94/treesMetadata/ (stored 0%)
  adding: bestModel/stages/14_RandomForestClassifier_06e8be3d9d94/treesMetadata/.part-00000-285bb550-015a-41d3-9dc7-2f17b91fdd03-c000.snappy.parquet.crc (stored 0%)
  adding: bestModel/stages/14_RandomForestClassifier_06e8be3d9d94/treesMetadata/part-00000-285bb550-015a-41d3-9dc7-2f17b91fdd03-c000.snappy.parquet (deflated 26%)
  adding: bestModel/stages/14_RandomForestClassifier_06e8be3d9d94/treesMetadata/_SUCCESS (stored 0%)
  adding: bestModel/

In [ ]:
pred=cvModel.transform(test)

In [ ]:
evaluator.evaluate(pred)

0.9467532467532467

# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.

In [ ]:
!python web_app.py

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/27 09:11:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/06/27 09:11:43 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://0f39faaa66ef280a7d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
23/06/27 09:26:35 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
23/06/27 12:03:51 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
23/06/27 12:03:52 ERROR Executor: Exception in task 1.0 in stage 64.0 (TID 66)
org.apache.spark.SparkException: [FAILED_EXECUTE_UDF] Failed to execute user defined function (String